CRISP DM Modell soll angewendet werden

In [438]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

In [439]:
df = pd.read_csv('titanic.csv', delimiter=",")

In [440]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [441]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


In [442]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [443]:
df_target = df['Survived']
df = df.drop(columns = ["Name", "Survived", "PassengerId"], axis = 1)

In [444]:
def fare_type(dataframe):
    dataframe['fare_amount'] = dataframe['Fare'].apply(lambda x: "high" if x > 15 else "low")
    return dataframe

In [445]:
def travel_alone(dataframe):
    dataframe['travel_alone'] = dataframe['SibSp'].apply(lambda x: 1 if x > 0 else 0)
    return dataframe


In [446]:
def travel_alone(dataframe):
    dataframe['travel_alone'] = dataframe['SibSp'].apply(lambda x: 1 if x > 0 else 0)
    return dataframe

In [447]:
# df = fare_type(df)
# df = travel_alone(df)
# df

In [448]:
encoder = LabelEncoder()
categories = ['Sex', 'Cabin', 'Ticket']

for i in categories:
    df[i] = encoder.fit_transform(df[i])

In [449]:
median_value = df['Age'].median() 
df['Age'] = df['Age'].fillna(median_value)

df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

C:\Users\dsaiger\AppData\Local\Temp\ipykernel_23924\446043650.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)


In [450]:
# Treating all the nominal variables at once using dummy variables
df_numeric=pd.get_dummies(df,dtype='int')

In [451]:
df = df_numeric
df

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S
0,3,1,22.0,1,0,523,7.2500,147,0,0,1
1,1,0,38.0,1,0,596,71.2833,81,1,0,0
2,3,0,26.0,0,0,669,7.9250,147,0,0,1
3,1,0,35.0,1,0,49,53.1000,55,0,0,1
4,3,1,35.0,0,0,472,8.0500,147,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,2,1,27.0,0,0,101,13.0000,147,0,0,1
887,1,0,19.0,0,0,14,30.0000,30,0,0,1
888,3,0,28.0,1,2,675,23.4500,147,0,0,1
889,1,1,26.0,0,0,8,30.0000,60,1,0,0


In [452]:
states = [27, 6728, 49122]
features = df
target = df_target

In [459]:
accs = []

for RANDOM_STATE in states:
    # Daten aufteilen
    X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size=0.3, random_state=RANDOM_STATE
    )
    
    # Skalieren
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Modell erstellen
    knn = KNeighborsClassifier(n_neighbors=20)
    
    # Cross-Validation durchführen
    cv_scores = cross_val_score(knn, X_train, y_train, cv=10)
    mean_cv_score = np.mean(cv_scores)
    
    # Modell trainieren und testen
    knn.fit(X_train, y_train)
    preds = knn.predict(X_test)
    test_accuracy = accuracy_score(y_test, preds)
    accs.append(test_accuracy)
    
    # Ergebnisse ausgeben
    print(f"Random State: {RANDOM_STATE}")
    print(f"Cross-Validation Accuracy (Train): {mean_cv_score:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(classification_report(y_test, preds, zero_division=0.0))

# Durchschnittliche Genauigkeit und Standardabweichung
accs_mean = round(np.mean(accs), 4)
accs_std = round(np.std(accs), 4)

print(f"Mean Test Accuracy: {accs_mean}")
print(f"Std Test Accuracy: {accs_std}")


Random State: 27
Cross-Validation Accuracy (Train): 0.8008
Test Accuracy: 0.8358
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       167
           1       0.82      0.72      0.77       101

    accuracy                           0.84       268
   macro avg       0.83      0.81      0.82       268
weighted avg       0.83      0.84      0.83       268

Random State: 6728
Cross-Validation Accuracy (Train): 0.7768
Test Accuracy: 0.8246
              precision    recall  f1-score   support

           0       0.85      0.88      0.86       168
           1       0.78      0.73      0.76       100

    accuracy                           0.82       268
   macro avg       0.82      0.81      0.81       268
weighted avg       0.82      0.82      0.82       268

Random State: 49122
Cross-Validation Accuracy (Train): 0.7785
Test Accuracy: 0.7985
              precision    recall  f1-score   support

           0       0.82      0.88      0.85